# UQ for an Optical Potential using ROSE

In [4]:
# (if you don't have it already)


In [10]:
# pip install nuclear-rose 

In [9]:
# pip install corner

In [8]:
# pip install surmise

In [11]:
import rose
import numpy as np
import time
import os
from scipy.stats import qmc
import matplotlib.pyplot as plt
import scipy.stats as sps
from tqdm import tqdm
import random
import datetime
import corner
import surmise





### Setting up the Scattering System

In [12]:
A = 40  # mass of the target
# asymptotic energy and wavenumber in center-of-mass (COM) frame
energy = 14  # MeV
# how many partial waves should we calculate?
l_max = 10
l_list = list(range(l_max + 1))
# domain of the differential cross section; the observable we want to emulate. We are setting them as equally spaced angles between 1 and 179 but this can be changed
angles = np.linspace(1, 179, 179)


### Next we either provide the experimental data we want to use, or a subset of angles and ROSE will generate mock data on them later

In [13]:
flag_provided_data =0  #Leave this as zero if user is only providing the "x" locations (the angles)



##################################################
#If providing data directly:
# angles_theta=
# anglesX=angles_theta- np.full(len(angles_theta), 1) #Moving the angles location by one because of how python works
# y_measured =   #Valueus of the observed cross section
# yerr=  #Errors of the observed cross section
# flag_provided_data=1    #Mark this as "1" if data is provided by user
##################################################


##################################################
#A subset of the angles in which we will deal with data
angles_theta=np.arange(20,160,5)
anglesX=angles_theta- np.full(len(angles_theta), 1) #Moving the angles location by one because of how python works

##################################################

### Finally some details and constants

In [14]:


AMU = 931.494102  # MeV/c^2, Particle Data Group
MASS_N = 1.008665 * AMU  # MeV/c^2 PDG
MASS_P = 1.007276 * AMU  # MeV/c^2 PDG
MASS_CHARGED_PION = 139.57039  # MeV/c^2
B_40CA = 342.0522  # BMEX

MASS_40CA = 20 * MASS_P + 20 * MASS_N - B_40CA
MU = (
    MASS_40CA * MASS_N / (MASS_40CA + MASS_N)
)  # reduced mass - we will do calculations in COM frame

#Alternatively you could define it as:
#MU = (AMU * A / (A+1)  ) 


k = np.sqrt(2 * MU * energy) / rose.constants.HBARC

rho=np.linspace(1e-6, 8*np.pi,2000)
s_0=6 * np.pi

## Defining the potentials

In [17]:
# shape of interaction for volume term
def wood_saxon(r, R, a):
    return 1 / (1 + np.exp((r - R) / a))


# shape of interaction for surface-peaked and spin-orbit coupling terms
def wood_saxon_prime(r, R, a):
    return -1 / a * np.exp((r - R) / a) / (1 + np.exp((r - R) / a)) ** 2


# total potential with a real and central term (with the same geometry)
# and imaginary surface-peaked term, but no SO coupling
def optical_potential(r, theta):
    Vv, Wv, Wd, Vso, Rv, Rd, Rso, av, ad, aso = theta
    return (-1j * Wv - Vv) * wood_saxon(r, Rv, av) + (4j * ad * Wd) * wood_saxon_prime(
        r, Rd, ad
    )


# spin orbit interaction constant
mso = rose.constants.HBARC / MASS_CHARGED_PION


# spin-orbit (SO) coulpling term - a function of l dot s, l being the orbital angular momentum
# and s being the spin of the neutron
def spin_orbit_potential(r, theta, ldots):
    Vv, Wv, Wd, Vso, Rv, Rd, Rso, av, ad, aso = theta
    return (Vso) * mso**2 * ldots * wood_saxon_prime(r, Rso, aso) / r


# the total number of parameters
nparams = 10

## Defining the parameters central value and the box around which we will train with the emulator

In [ ]:
# This is the value of the parameters coming from the Koning-Delaroche parametrization.
# Taken from https://www-nds.iaea.org/RIPL-3/
VvKD = 48.9
WvKD = 1.2
WdKD = 7.7
VsoKD = 5.5


RvKD = 1.19* 40 ** (1.0 / 3.0)
RdKD = 1.29* 40 ** (1.0 / 3.0)
RsoKD = 1.00 * 40 ** (1.0 / 3.0)

avKD = 0.67
adKD = 0.67
asoKD = 0.59

alphaKD = np.array(
    [VvKD, WvKD, WdKD, VsoKD, RvKD, RdKD, RsoKD, avKD, adKD, asoKD]
)



#The following are the center of the parameters to train the emulator
Vv0 = 45
Wv0 = 2
Wd0 = 5
Vso = 5



Rv0 = 4

Rd0 = 4
Rso = 4

av0 = 0.5

ad0 = 0.5
aso = 0.5


#This is the center of our prior and around where we will train our RBM emulator
alphaCentral = np.array([Vv0, Wv0, Wd0, Vso, Rv0, Rd0, Rso, av0 , ad0, aso])





scaleTraining = 0.3

bounds = np.array(
    [
        alphaCentral - np.fabs(alphaCentral * scaleTraining),
        alphaCentral + np.fabs(alphaCentral * scaleTraining),
    ]
).T


def sample_points(npoints, bounds,initial_seed=None):
    sampler = qmc.LatinHypercube(d=len(bounds), seed=initial_seed)
    sample = sampler.random(npoints)
    scaled = qmc.scale(sample, bounds[:, 0], bounds[:, 1])
    return scaled